# Scrap table from url

In [2]:
from bs4 import BeautifulSoup
import requests

# Scrape data table from URL
url = 'https://www.espncricinfo.com/records/highest-career-batting-average-282910'

# Change User-Agent
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link crossorigin="anonymous" href="https://img1.hscicdn.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://dcf.espn.com" rel="preconnect"/>
  <link as="font" crossorigin="anonymous" href="https://wassets.hscicdn.com/static/fonts/CiIcons/ci-icons-v3.5/fonts/icomoon.woff?gencn3" rel="preload" type="font/woff2"/>
  <script type="text/javascript">
   /*! js-cookie v3.0.5 | MIT */
          !function(e,t){"object"==typeof exports&&"undefined"!=typeof module?module.exports=t():"function"==typeof define&&define.amd?define(t):(e="undefined"!=typeof globalThis?globalThis:e||self,function(){var n=e.Cookies,o=e.Cookies=t();o.noConflict=function(){return e.Cookies=n,o}}())}(this,(function(){"use strict";function e(e){for(var t=1;t<arguments.length;t++){var n=arguments[t];for(var o in n)e[o]=n[o]}return e}var t=function t(n,o){function r(t,r,i){if("undefined"!=typeof document){"number"==typeof(i=e({},o,i)).expi

In [3]:
# Soup inspection
table_element = soup.find("table")
print(table_element.prettify())


<table class="ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide">
 <thead class="ds-bg-fill-content-alternate ds-text-left">
  <tr class="">
   <td class="ds-min-w-max">
    <div class="ds-popper-wrapper">
     <span class="ds-cursor-pointer">
      Player
     </span>
    </div>
   </td>
   <td class="ds-min-w-max ds-text-right">
    <div class="ds-popper-wrapper">
     <span class="ds-cursor-pointer">
      Span
     </span>
    </div>
   </td>
   <td class="ds-min-w-max ds-text-right">
    <div class="ds-popper-wrapper">
     <span class="ds-cursor-pointer">
      Mat
     </span>
    </div>
   </td>
   <td class="ds-min-w-max ds-text-right">
    <div class="ds-popper-wrapper">
     <span class="ds-cursor-pointer">
      Inns
     </span>
    </div>
   </td>
   <td class="ds-min-w-max ds-text-right">
    <div class="ds-popper-wrapper">
     <span class="ds-cursor-pointer">
      NO
     </span>
    </div>
   </td>
   <td class="ds-min-w-max 

In [4]:
# Locate the headers
first_row = table_element.find('tr')  
titles = first_row.find_all('td') # Extract the first row's cells
headers = [title.text.strip() for title in titles]
print(headers)

# Dump header into dataframe
import pandas as pd
df = pd.DataFrame(columns=headers)
df

['Player', 'Span', 'Mat', 'Inns', 'NO', 'Runs', 'HS', 'Ave', 'BF', 'SR', '100', '50', '0', '4s', '6s']


,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s


In [5]:
row_data = table_element.find_all('tr')[1:]

for row in row_data:
    player_data = row.find_all('td')
    indi_player = [player.text.strip() for player in player_data]
    length = len(df)
    df.loc[length] = indi_player
df
    

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s
0,DG Bradman (AUS),1928-1948,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6
1,PHKD Mendis (SL),2022-2025,12,21,2,1184,182*,62.31,1839,64.38,5,4,-,123,23
2,AC Voges (AUS),2015-2016,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5
3,RG Pollock (SA),1963-1970,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11
4,GA Headley (WI),1930-1954,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,KD Walters (AUS),1965-1981,74,125,14,5357,250,48.26,8662+,49.16,15,33,4,525+,23
58,GC Smith (ICC/SA),2002-2014,117,205,13,9265,277,48.25,15525,59.67,27,38,11,1165,24
59,WH Ponsford (AUS),1924-1934,29,48,4,2122,266,48.22,3118+,44.77,7,6,1,119+,0
60,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+


# Some alternation (duplication, null value)

In [7]:
# add duplicates, null values
df.replace("-", None, inplace=True)
# Find rows where "Player" contains
gs_rows = df[df["Player"].str.contains("GS Sobers")]  
jb_rows = df[df["Player"].str.contains("JB")]
sj_rows = df[df["Player"].str.contains("SJ McCabe")]
# Duplicate those rows and append to DataFrame
df = pd.concat([df, gs_rows], ignore_index=True)
df = pd.concat([df, jb_rows], ignore_index=True)
df = pd.concat([df, sj_rows], ignore_index=True)
# Display the updated DataFrame
print(df)

              Player       Span Mat Inns  NO  Runs    HS    Ave     BF     SR  \
0   DG Bradman (AUS)  1928-1948  52   80  10  6996   334  99.94  9800+  58.60   
1   PHKD Mendis (SL)  2022-2025  12   21   2  1184  182*  62.31   1839  64.38   
2     AC Voges (AUS)  2015-2016  20   31   7  1485  269*  61.87   2667  55.68   
3    RG Pollock (SA)  1963-1970  23   41   4  2256   274  60.97  1707+  54.48   
4    GA Headley (WI)  1930-1954  22   40   4  2190  270*  60.83   416+  56.00   
..               ...        ...  ..  ...  ..   ...   ...    ...    ...    ...   
60   SJ McCabe (AUS)  1930-1938  39   62   5  2748   232  48.21  3217+  60.02   
61  DR Jardine (ENG)  1928-1934  22   33   6  1296   127  48.00  2110+  25.59   
62    GS Sobers (WI)  1954-1974  93  160  21  8032  365*  57.78  4063+  53.58   
63    JB Hobbs (ENG)  1908-1930  61  102   7  5410   211  56.94  5363+  46.22   
64   SJ McCabe (AUS)  1930-1938  39   62   5  2748   232  48.21  3217+  60.02   

   100  50     0    4s   6s

In [ ]:
# See all rows

# 1. Rename

In [9]:
# Rename multiple columns name in a list

In [10]:
df = df.rename(columns={'Mat':'Matches', 'NO':'Not_Outs', 'HS':'Highest_Inns_Score', 'BF':'Ball_Faced', 'SR':'Batting_Strike_rate'})
df

,Player,Span,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s
0,DG Bradman (AUS),1928-1948,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6
1,PHKD Mendis (SL),2022-2025,12,21,2,1184,182*,62.31,1839,64.38,5,4,None,123,23
2,AC Voges (AUS),2015-2016,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5
3,RG Pollock (SA),1963-1970,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11
4,GA Headley (WI),1930-1954,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+
61,DR Jardine (ENG),1928-1934,22,33,6,1296,127,48.00,2110+,25.59,1,10,2,53+,0
62,GS Sobers (WI),1954-1974,93,160,21,8032,365*,57.78,4063+,53.58,26,30,12,593+,32+
63,JB Hobbs (ENG),1908-1930,61,102,7,5410,211,56.94,5363+,46.22,15,28,4,276+,8


# 2. isna, fillna

In [12]:
# Check for null values

In [13]:
# null inspection
df.isna().any()

Player                 False
Span                   False
Matches                False
Inns                   False
Not_Outs               False
Runs                   False
Highest_Inns_Score     False
Ave                    False
Ball_Faced              True
Batting_Strike_rate    False
100                    False
50                     False
0                       True
4s                     False
6s                     False
dtype: bool

In [14]:
# Show all the null row
df[df['Ball_Faced'].isna()==1]

,Player,Span,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s
8,ED Weekes (WI),1948-1958,48,81,5,4455,207,58.61,None,0.00,15,19,6,258+,2
15,CL Walcott (WI),1948-1960,44,74,7,3798,220,56.68,None,0.00,15,14,1,107+,11
54,Hon.FS Jackson (ENG),1893-1905,20,33,4,1415,144*,48.79,None,0.00,5,6,3,51+,0


In [15]:
# Fill in nah with 0 separately by column
df['Ball_Faced'] = df['Ball_Faced'].fillna(0)
df['0'] = df['0'].fillna(0)
# Fill in nah with 0 for whole table
df.fillna(0, inplace=True)

df.isna().any()

Player                 False
Span                   False
Matches                False
Inns                   False
Not_Outs               False
Runs                   False
Highest_Inns_Score     False
Ave                    False
Ball_Faced             False
Batting_Strike_rate    False
100                    False
50                     False
0                      False
4s                     False
6s                     False
dtype: bool

# 3. Duplicate

In [17]:
# Check duplicate

In [18]:
df.duplicated()

0     False
1     False
2     False
3     False
4     False
      ...  
60    False
61    False
62     True
63     True
64     True
Length: 65, dtype: bool

In [19]:
# Line out duplicates

In [20]:
df[df['Player'].duplicated()==1]

,Player,Span,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s
62,GS Sobers (WI),1954-1974,93,160,21,8032,365*,57.78,4063+,53.58,26,30,12,593+,32+
63,JB Hobbs (ENG),1908-1930,61,102,7,5410,211,56.94,5363+,46.22,15,28,4,276+,8
64,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+


In [21]:
# Line out ALL duplicates

In [22]:
df[df['Player'].isin(['GS Sobers (WI)', 'JB Hobbs (ENG)', 'SJ McCabe (AUS)'])]

,Player,Span,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s
11,GS Sobers (WI),1954-1974,93,160,21,8032,365*,57.78,4063+,53.58,26,30,12,593+,32+
13,JB Hobbs (ENG),1908-1930,61,102,7,5410,211,56.94,5363+,46.22,15,28,4,276+,8
60,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+
62,GS Sobers (WI),1954-1974,93,160,21,8032,365*,57.78,4063+,53.58,26,30,12,593+,32+
63,JB Hobbs (ENG),1908-1930,61,102,7,5410,211,56.94,5363+,46.22,15,28,4,276+,8
64,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+


In [23]:
# Drop duplicate

In [24]:
df = df.drop_duplicates()  # Or inplace=True

In [25]:
df[df['Player'].isin(['GS Sobers (WI)', 'JB Hobbs (ENG)', 'SJ McCabe (AUS)'])]

,Player,Span,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s
11,GS Sobers (WI),1954-1974,93,160,21,8032,365*,57.78,4063+,53.58,26,30,12,593+,32+
13,JB Hobbs (ENG),1908-1930,61,102,7,5410,211,56.94,5363+,46.22,15,28,4,276+,8
60,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+


# 4. Split

In [27]:
# Split span column into start and end year

In [28]:
df['Rookie_Year'] = df['Span'].str.split(pat='-').str[0]
df['Final_Year'] = df['Span'].str.split(pat='-').str[1]
df

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/1010708635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rookie_Year'] = df['Span'].str.split(pat='-').str[0]
/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/1010708635.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Final_Year'] = df['Span'].str.split(pat='-').str[1]


,Player,Span,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s,Rookie_Year,Final_Year
0,DG Bradman (AUS),1928-1948,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6,1928,1948
1,PHKD Mendis (SL),2022-2025,12,21,2,1184,182*,62.31,1839,64.38,5,4,0,123,23,2022,2025
2,AC Voges (AUS),2015-2016,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5,2015,2016
3,RG Pollock (SA),1963-1970,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11,1963,1970
4,GA Headley (WI),1930-1954,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1,1930,1954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,KD Walters (AUS),1965-1981,74,125,14,5357,250,48.26,8662+,49.16,15,33,4,525+,23,1965,1981
58,GC Smith (ICC/SA),2002-2014,117,205,13,9265,277,48.25,15525,59.67,27,38,11,1165,24,2002,2014
59,WH Ponsford (AUS),1924-1934,29,48,4,2122,266,48.22,3118+,44.77,7,6,1,119+,0,1924,1934
60,SJ McCabe (AUS),1930-1938,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+,1930,1938


In [29]:
# Drop Span

In [30]:
df.drop(['Span'], axis=1, inplace=True)

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/2580495033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Span'], axis=1, inplace=True)


In [68]:
# Split the country abbrev out of player column

In [66]:
df['Player'].str.split(pat='(')
df['Country'] = df['Player'].str.split(pat='(').str[1]
df['Country'] = df['Country'].str.split(pat=')').str[0]
df['Player'] = df['Player'].str.split(pat='(').str[0]
df

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/3643439797.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Player'].str.split(pat='(').str[1]
/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/3643439797.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Country'] = df['Country'].str.split(pat=')').str[0]
/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/3643439797.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

,Player,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s,Rookie_Year,Final_Year,Country
0,DG Bradman,52,80,10,6996,334,99.94,9800+,58.60,29,13,7,626+,6,1928,1948,AUS
1,PHKD Mendis,12,21,2,1184,182*,62.31,1839,64.38,5,4,0,123,23,2022,2025,SL
2,AC Voges,20,31,7,1485,269*,61.87,2667,55.68,5,4,2,186,5,2015,2016,AUS
3,RG Pollock,23,41,4,2256,274,60.97,1707+,54.48,7,11,1,246+,11,1963,1970,SA
4,GA Headley,22,40,4,2190,270*,60.83,416+,56.00,10,5,2,104+,1,1930,1954,WI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,KD Walters,74,125,14,5357,250,48.26,8662+,49.16,15,33,4,525+,23,1965,1981,AUS
58,GC Smith,117,205,13,9265,277,48.25,15525,59.67,27,38,11,1165,24,2002,2014,ICC/SA
59,WH Ponsford,29,48,4,2122,266,48.22,3118+,44.77,7,6,1,119+,0,1924,1934,AUS
60,SJ McCabe,39,62,5,2748,232,48.21,3217+,60.02,6,13,4,241+,5+,1930,1938,AUS


# 5. Change Data Type

In [ ]:
# Inspect the dtypes

In [163]:
df.dtypes

Player                  object
Matches                  int64
Inns                     int64
Not_Outs                 int64
Runs                     int64
Highest_Inns_Score       int64
Ave                    float64
Ball_Faced               int64
Batting_Strike_rate    float64
100                      int64
50                       int64
0                        int64
4s                       int64
6s                       int64
Rookie_Year              int64
Final_Year               int64
Country                 object
dtype: object

In [149]:
df.head()

,Player,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s,Rookie_Year,Final_Year,Country
0,DG Bradman,52,80,10,6996,334,99.94,9800,58.60,29,13,7,626+,6,1928,1948,AUS
1,PHKD Mendis,12,21,2,1184,182,62.31,1839,64.38,5,4,0,123,23,2022,2025,SL
2,AC Voges,20,31,7,1485,269,61.87,2667,55.68,5,4,2,186,5,2015,2016,AUS
3,RG Pollock,23,41,4,2256,274,60.97,1707,54.48,7,11,1,246+,11,1963,1970,SA
4,GA Headley,22,40,4,2190,270,60.83,416,56.00,10,5,2,104+,1,1930,1954,WI


In [129]:
df['Ball_Faced'] = df['Ball_Faced'].str.split(pat='+').str[0]

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/3892440399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ball_Faced'] = df['Ball_Faced'].str.split(pat='+').str[0]


In [145]:
df['Ball_Faced'] = df['Ball_Faced'].astype('int')

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/195800819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ball_Faced'] = df['Ball_Faced'].astype('int')


In [141]:
df['Ball_Faced'].isna().any()

False

In [143]:
df[df['Ball_Faced'].isna()==1]

,Player,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s,Rookie_Year,Final_Year,Country


In [139]:
df['Ball_Faced'].fillna(0, inplace=True)

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/1108331108.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ball_Faced'].fillna(0, inplace=True)


# Calculation

In [ ]:
# Create a new career length column

In [168]:
df['Career_length'] = df['Final_Year'] - df['Rookie_Year']

/var/folders/8f/cc_txdtd2czdwfj58rytzkth0000gn/T/ipykernel_96141/4195033281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Career_length'] = df['Final_Year'] - df['Rookie_Year']


In [172]:
# 1. What's the average career year?

In [180]:
df['Career_length'].mean().round(2)

12.97

In [184]:
# 2. AVG Batting_Strike_rate for players who played over 10 years

In [192]:
df[df['Career_length']>10]['Batting_Strike_rate'].mean()

47.80186046511628

In [ ]:
# 3. Find num of players who played before 1960

In [200]:
df[df['Final_Year']<1960]['Player'].count()

18

In [186]:
# 4.Max Highest_Inns_Score by country

In [218]:
df.groupby('Country')['Highest_Inns_Score'].max()

Country
AUS        380
ENG        364
ICC/IND    319
ICC/PAK    329
ICC/SA     277
ICC/WI     400
IND        248
NZ         251
PAK        313
SA         278
SL         374
WI         365
ZIM        232
Name: Highest_Inns_Score, dtype: int64

In [ ]:
# 5.100, 50, 0 AVG by country

In [224]:
df.groupby('Country')[['100', '50', '0']].mean()

,100,50,0
Country,,,
AUS,21.533333,28.933333,8.933333
ENG,13.076923,21.384615,4.461538
ICC/IND,29.500000,47.500000,12.000000
ICC/PAK,25.000000,46.000000,15.000000
ICC/SA,36.000000,48.000000,13.500000
ICC/WI,34.000000,48.000000,17.000000
IND,23.250000,31.500000,8.000000
NZ,33.000000,37.000000,11.000000
PAK,17.800000,24.200000,8.000000


In [182]:
df.head()

,Player,Matches,Inns,Not_Outs,Runs,Highest_Inns_Score,Ave,Ball_Faced,Batting_Strike_rate,100,50,0,4s,6s,Rookie_Year,Final_Year,Country,Career_length
0,DG Bradman,52,80,10,6996,334,99.94,9800,58.60,29,13,7,626,6,1928,1948,AUS,20
1,PHKD Mendis,12,21,2,1184,182,62.31,1839,64.38,5,4,0,123,23,2022,2025,SL,3
2,AC Voges,20,31,7,1485,269,61.87,2667,55.68,5,4,2,186,5,2015,2016,AUS,1
3,RG Pollock,23,41,4,2256,274,60.97,1707,54.48,7,11,1,246,11,1963,1970,SA,7
4,GA Headley,22,40,4,2190,270,60.83,416,56.00,10,5,2,104,1,1930,1954,WI,24


In [1]:
df[df['Player'].str.contains('DG Bradman', case=False, na=False)]

NameError: name 'df' is not defined

In [33]:
Great question! If you don’t know which specific column contains 'DG Bradman', you can search the entire DataFrame.

🔹 Solution: Find 'DG Bradman' Anywhere in df

df[df.apply(lambda row: row.astype(str).str.contains('DG Bradman', na=False).any(), axis=1)]

SyntaxError: invalid character '’' (U+2019) (1291726386.py, line 1)